# Comparador B < A por Identificación

## 1) Configuración

In [1]:

import pandas as pd
import re, unicodedata, os

# Rutas (ajusta a tu entorno)
excel_a = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Comparacion2025A\data\2025A.xlsx"
excel_b = r"C:\Users\andra\Documents\8vo\Vice\automatizacion\ProyectoPreplanificacion\Comparacion2025A\data\2025B.xlsx"

# Parámetros
IDENT_H = "Identificación"
CLS_H   = "Cls-Dictadas"
EPS     = 1e-9

# Observaciones (se añade igual para todas las filas del reporte final)
OBS_TEXT = (
    "Se observa una disminución en la carga\n"
    "horaria de clases en el periodo 2025-B. Se recuerda que dicha\n"
    "reducción solo será válida en caso de que el\n"
    "departamento no solicite la incorporación de nuevo\n"
    "personal académico."
)

# Columnas a guardar (con duplicado de 'Profesor', queremos la **segunda**)
COLS_PEDIDAS = [
    "Período Académico","Departamento","Tipo Cargo","Dedicación","Profesor","Identificación",
    "Profesor","Tipo","Cargo","Dedicación","Fecha Inicio","Fecha Fin","Estado","Horas Exigibles"
]

# Salida
out_dir   = "outputs_notebook"
out_xlsx  = "Reporte_B_menor_A.xlsx"
aplicar_formato_excel = True  # auto-filtros + encabezado congelado + ajuste simple de anchos


## 2) Utilidades

In [2]:

def _norm_text(s: str) -> str:
    """acentos fuera, minúsculas, recorta"""
    return "".join(ch for ch in unicodedata.normalize("NFKD", str(s)) if not unicodedata.combining(ch)).lower().strip()

def normalize_ident(v: str) -> str:
    s = _norm_text(v).replace(" ", "").replace("\u00A0", "")
    digits = re.sub(r"\D", "", s)
    return digits if len(digits) >= 8 else re.sub(r"[^0-9a-z]", "", s)

def numify(series: pd.Series) -> pd.Series:
    def _norm(s):
        s = str(s).replace(" ", "").replace("\u00A0", "")
        s = re.sub(r"[^0-9,.\-]", "", s)
        if s.count(",") and not s.count("."): s = s.replace(",", ".")
        elif s.count(",") and s.count("."):    s = s.replace(".", "").replace(",", ".")
        return s
    return pd.to_numeric(series.astype(str).map(_norm), errors="coerce").fillna(0)

def read_master(path: str) -> pd.DataFrame:
    """Lee todas las hojas a dtype=object y agrega columna _sheet."""
    xls = pd.ExcelFile(path)  # requiere openpyxl instalado
    dfs = []
    for sh in xls.sheet_names:
        df = pd.read_excel(path, sheet_name=sh, dtype="object")
        df.columns = [_norm_text(c) for c in df.columns]
        df["_sheet"] = sh
        dfs.append(df)
    out = pd.concat(dfs, ignore_index=True)
    out.insert(0, "row_index", out.index.astype(int))
    return out

def resolve_col(df: pd.DataFrame, name: str) -> str:
    """Retorna la primera columna donde 'name' (normalizado) es substring."""
    tgt = _norm_text(name)
    for c in df.columns:
        if tgt in c: return c
    raise ValueError(f"No encontrada: {name} → '{tgt}'. Columnas: {df.columns[:10].tolist()} ...")


## 3) Comparar B < A y exportar 

In [3]:

def comparar_y_exportar(excel_a: str, excel_b: str):
    A = read_master(excel_a)
    B = read_master(excel_b)

    id_a, cls_a = resolve_col(A, IDENT_H), resolve_col(A, CLS_H)
    id_b, cls_b = resolve_col(B, IDENT_H), resolve_col(B, CLS_H)

    # Sumar horas por identificación
    A_ = A[[id_a, cls_a]].rename(columns={id_a: "ident", cls_a: "cls"})
    A_["ident"] = A_["ident"].map(normalize_ident)
    A_["cls"]   = numify(A_["cls"])
    a_sum = A_.groupby("ident")["cls"].sum()

    B_ = B[[id_b, cls_b] + [c for c in B.columns if c not in [id_b, cls_b]]].copy()
    B_["ident_norm"] = B_[id_b].map(normalize_ident)
    B_["cls"]        = numify(B_[cls_b])
    b_sum = B_.groupby("ident_norm")["cls"].sum()

    # Intersección y condición B < A
    a_al, b_al = a_sum.align(b_sum, join="inner")
    mask = (b_al + EPS) < a_al
    diffs = pd.DataFrame({
        "ident_norm": a_al.index[mask],
        "cls_A": a_al[mask].values,
        "cls_B": b_al[mask].values
    })
    diffs["diferencia"] = diffs["cls_A"] - diffs["cls_B"]

    # Metadatos de B (una fila por ID) y unión
    meta = B_.drop_duplicates("ident_norm")
    out  = meta.merge(diffs, left_on="ident_norm", right_on="ident_norm", how="inner")

    # Orden por departamento si existe
    dept_cols = [c for c in out.columns if "departamento" in c]
    if dept_cols:
        out = out.sort_values(by=dept_cols[0], kind="stable")

    # Observaciones
    out["Observaciones"] = OBS_TEXT

    # === Selección: solo columnas pedidas (con 2do 'Profesor') + extras ===
    cols_norm = [_norm_text(c) for c in out.columns]
    extras    = ["cls_A","cls_B","diferencia","Observaciones"]  # se agregan al final si existen
    selected_idx = []
    selected_labels = []

    # Ubicar la segunda ocurrencia de 'profesor' en out
    prof_target = _norm_text("Profesor")
    prof_positions = [i for i, c in enumerate(cols_norm) if prof_target in c]
    prof2_idx = prof_positions[1] if len(prof_positions) >= 2 else (prof_positions[0] if prof_positions else None)

    for h in COLS_PEDIDAS:
        tgt = _norm_text(h)
        if tgt == prof_target:
            if prof2_idx is not None:
                selected_idx.append(prof2_idx)
                selected_labels.append(h)  # mantenemos el rótulo 'Profesor'
            # si no hay segunda, se omite silenciosamente
            continue
        # primera coincidencia normal
        idx = next((i for i, c in enumerate(cols_norm) if tgt in c), None)
        if idx is not None:
            selected_idx.append(idx)
            selected_labels.append(h)

    # Construir DataFrame final ordenado
    base_cols = [out.columns[i] for i in selected_idx]
    final_cols = base_cols + [c for c in extras if c in out.columns]
    df_save = out[final_cols].copy()

    # Renombrar a etiquetas humanas (para las pedidas); extras se quedan igual
    rename_map = {col: label for col, label in zip(base_cols, selected_labels)}
    df_save = df_save.rename(columns=rename_map)

    # Exportar Excel con filtros
    os.makedirs(out_dir, exist_ok=True)
    xlsx_path = os.path.join(out_dir, out_xlsx)
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xl:
        df_save.to_excel(xl, sheet_name="B_menor_A", index=False)
        if aplicar_formato_excel:
            from openpyxl.styles import Font, Alignment
            wb = xl.book; ws = wb["B_menor_A"]

            # encabezados en negrita + centrados
            for cell in next(ws.iter_rows(min_row=1, max_row=1)):
                cell.font = Font(bold=True)
                cell.alignment = Alignment(horizontal="center")

            ws.auto_filter.ref = ws.dimensions
            ws.freeze_panes = "A2"

            headers = {c.value: i+1 for i, c in enumerate(ws[1])}
            for col in ws.columns:
                maxlen = max((len(str(c.value)) if c.value is not None else 0) for c in col)
                ws.column_dimensions[col[0].column_letter].width = min(max(12, maxlen + 2), 60)

            if "Observaciones" in headers:
                col_letter = ws.cell(row=1, column=headers["Observaciones"]).column_letter
                for cell in ws[col_letter][1:]:
                    cell.alignment = Alignment(wrap_text=True, vertical="top")

    print(f"OK. Excel generado: {xlsx_path}")
    return df_save

df_final = comparar_y_exportar(excel_a, excel_b)
df_final.head(10)


c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


OK. Excel generado: outputs_notebook\Reporte_B_menor_A.xlsx


,Período Académico,Departamento,Cargo,Dedicación,Profesor,Identificación,Profesor,Cargo,Cargo,Dedicación,Fecha Inicio,Fecha Fin,Estado,Horas Exigibles,cls_A,cls_B,diferencia,Observaciones
72,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,TRUJILLO GUERRERO MARIA FERNANDA,0604024158,TRUJILLO GUERRERO MARIA FERNANDA,TITULAR,TITULAR,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,REVISION INICIAL,976,276.0,234.0,42.0,Se observa una disminución en la carga\nhorari...
73,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,RIVERA ARGOTI PABLO ROBINSON,1705981981,RIVERA ARGOTI PABLO ROBINSON,TITULAR,TITULAR,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,REVISION INICIAL,976,216.0,180.0,36.0,Se observa una disminución en la carga\nhorari...
74,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,GAMBOA BENITEZ SILVANA DEL PILAR,1712333721,GAMBOA BENITEZ SILVANA DEL PILAR,TITULAR,TITULAR,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,POR RECTIFICAR INICIAL,976,262.0,198.0,64.0,Se observa una disminución en la carga\nhorari...
75,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,CRUZ DAVALOS PATRICIO JAVIER,1713199493,CRUZ DAVALOS PATRICIO JAVIER,TITULAR,TITULAR,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,POR RECTIFICAR INICIAL,976,180.0,126.0,54.0,Se observa una disminución en la carga\nhorari...
76,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,CHAVEZ GARCIA GEOVANNY DANILO,1713913752,CHAVEZ GARCIA GEOVANNY DANILO,TITULAR,TITULAR,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,PLANIFICADO INICIAL,976,154.0,126.0,28.0,Se observa una disminución en la carga\nhorari...
77,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,MOYA GONZALEZ VIVIANA ISABEL,1724424617,MOYA GONZALEZ VIVIANA ISABEL,TITULAR,TITULAR,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,POR RECTIFICAR INICIAL,976,264.0,198.0,66.0,Se observa una disminución en la carga\nhorari...
78,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TÉCNICO DOCENTE POLITÉCNICO,TC,CHUQUI GAVILANES JOHNNY MAURICIO,0503631350,CHUQUI GAVILANES JOHNNY MAURICIO,TÉCNICO DOCENTE POLITÉCNICO,TÉCNICO DOCENTE POLITÉCNICO,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,POR RECTIFICAR INICIAL,976,396.0,324.0,72.0,Se observa una disminución en la carga\nhorari...
79,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TÉCNICO DOCENTE POLITÉCNICO,TC,VEGA PINTO JORGE LUIS,1720037553,VEGA PINTO JORGE LUIS,TÉCNICO DOCENTE POLITÉCNICO,TÉCNICO DOCENTE POLITÉCNICO,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,REVISION INICIAL,976,360.0,342.0,18.0,Se observa una disminución en la carga\nhorari...
80,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TÉCNICO DOCENTE POLITÉCNICO,TC,DIAZ CAJAS VICTOR RENATO,1720136637,DIAZ CAJAS VICTOR RENATO,TÉCNICO DOCENTE POLITÉCNICO,TÉCNICO DOCENTE POLITÉCNICO,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,PLANIFICADO INICIAL,976,360.0,324.0,36.0,Se observa una disminución en la carga\nhorari...
81,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TÉCNICO DOCENTE POLITÉCNICO,TC,CUAYCAL BASTIDAS ANDRES FERNANDO,1720633708,CUAYCAL BASTIDAS ANDRES FERNANDO,TÉCNICO DOCENTE POLITÉCNICO,TÉCNICO DOCENTE POLITÉCNICO,TC,2025-09-01 00:00:00,2026-02-28 00:00:00,POR RECTIFICAR INICIAL,976,396.0,342.0,54.0,Se observa una disminución en la carga\nhorari...
